## Example sPlot  to create signal weights to be used in subsequent fit

The weights will be stored in a ROOT file as part of a HS::Weights object. They can then be combined with the data on the fly when performing subsequent fits. It is also possible to merge with the data tree :
    
       Weights wgts;
       wgts.LoadSaved("filename.root","HSsWeights");
       wgts.AddToTree(tree);
       
If this is attempted in interactive ROOT you will also need to have loaded the classes (next line)

First load the hs fit classes

In [1]:
gROOT->ProcessLine(".x $HSCODE/hsfit/LoadFit.C+")

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ FiledTree
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Bins
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Weights


Info in <TUnixSystem::ACLiC>: creating shared library /work/Dropbox/HaSpect/dev/HASPECT6/hsdata/Weights_C.so
Warning in cling::IncrementalParser::CheckABICompatibility():
  Possible C++ standard library mismatch, compiled with __GLIBCXX__ '20180415'
  Extraction of runtime standard library version was: '20180720'


^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ RooHSEventsPDF

RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ RooHSEventsHistPDF
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Setup
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Data
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Binner
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Minimiser
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ HSMetropolisHastings
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ RooMcmc
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ PlotResults
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ BootStrapper
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ FitManager
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ sPlot


Info in <TUnixSystem::ACLiC>: creating shared library /work/Dropbox/HaSpect/dev/HASPECT6/hsfit/sPlot_C.so
Warning in cling::IncrementalParser::CheckABICompatibility():
  Possible C++ standard library mismatch, compiled with __GLIBCXX__ '20180415'
  Extraction of runtime standard library version was: '20180720'


^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ FitSelector
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Process
(long) 0


### Construct an sPlot object for running the fit

Set an output directory for saving weights and plots and stdout. It is best to provide the full path as this can then be used with PROOF as well.

I use the pwd variable to keep everything in the local directory, while applying the full path for PROOF.

In [2]:
sPlot RF;
TString pwd = TString(gSystem->Getenv("PWD"))+"/";
RF.SetUp().SetOutDir(pwd+"sPlotFit");

### Set an observables

These are the fit variables. In general you can fit in as many dimensions as you need by supplying more variables. 

Here we are going to fit the missing mass (Mmiss) between 0-10


In [3]:
///////////////////////////////Load Variables
RF.SetUp().LoadVariable("Mmiss[0,10]");//should be same name as variable in tree

IMPORTANT here we set the event ID variable. Each event in your tree should have a unique id with which to synchronise with weights.
    This is useful as it allows us to break the data up, perform seperate fits then combine the weights afterwards.
    The weights are not written into the input tree, rather they are stored seperately (as HS::Weights) and combined when required. This allows you to use various sets of weights when performing fits or regenerate the weights after some corrections etc.

In [4]:
RF.SetUp().SetIDBranchName("fgID"); //this is set in Model1.C

### Create a signal PDF

The LoadSpecies Line adds the PDF Signal to the Extended Maximum Likelihood fit and creates a corresponding yield parameter Yld_Signal which will give the number of signal events in the data after the fit.
Note, I did not have to directly load this PDF I could have created more PDFs and combined them to make a more complicated PDF shape first.

We are just going to use a signal Gaussain with initial guess of mean 4 (with possible range 2-8), and sigma 1 (range 0.1-5)

In [5]:
//////////////////////////////Make signal PDF
RF.SetUp().FactoryPDF("Gaussian::Signal( Mmiss, mean[4,2,8], sigma[1,0.1,5] )");
RF.SetUp().LoadSpeciesPDF("Signal",1);

### Create a background PDF

Here we just use a quadratic Chebychev polynomial to describe a smooth background. You can add extra a2,a3,etc. to increase the degree of the polynomial

In [6]:
//////////////////////////////Make background PDF
RF.SetUp().FactoryPDF("Chebychev::BG(Mmiss,{a0[-0.1,-1,1],a1[0.1,-1,1]})");
RF.SetUp().LoadSpeciesPDF("BG",1);

### Define Bins/Splits
Often you wil want to split the data into distinct bins, in invariant mass or angle for example, and perform seperate fits to each bin. Here we can set as many different variables to split the data on as we like. Here we must use the Binner class accessed via RF.Bins() to propogate the splits into data and simulated events.

The function LoadBinVars takes a variable to make bins out of which has to be in the data tree, the number of bins to make and the limits of the variable. Alternatively you can supply the variable name, number of bins and an array of bin edges as for TAxis.

In this example we will not use splits, but you can test it by commenting out the line below. In this case after performing the individaul fits all weights will be combined into 1 file OUTDIR/Tweights.root

In [7]:
//RF.Bins().LoadBinVar("Eg",5,3,4);

### Load Data


This just takes the treename "MyModel" and data file "Data.root" which were created in GenerateData

In [8]:
///////////////////////////Load Data
RF.LoadData("MyModel","Data.root");

DataEvents::Load MyModel 1


Turn on javascript root for interactive plots (even when saved as html file!)

In [9]:
%jsroot

### Run the fit.
The test output will be redirected to your specified output directory (see code cell 2) as file logRooFit.txt. This will give full details of the fit.
Here we will just report the final result, the weights and the plots of fit PDF overlayed on the data as well as the fit residuals and pulls.

There are different options for running. 
To just run standard use Here::Go and the fits will just be performed within this interactive session, sequentially in the case of bins.

Here::One(&RF,i) will run one singal fit from the different bins produce with LoadBinVars, where i gives the index of the bin

Use Proof::Go(&RF,N) and the fits will be run in parallel via PROOF with each split/bin running on a different core, where N = number of cores to use.
    

In [10]:
Here::Go(&RF);
//Here::One(&RF,3);
//Proof::Go(&RF,1);

FitManager::RunOne() 0 /work/Dropbox/HaSpect/dev/HASPECT6/tutorials/RooFitExamples/WeightedObservable/sPlotFit/
HS::FIT::sPlot::Do prelimanry fits PyROOT
%%%%%%%%%%%%%%%%%%%%%copy setup 
FitManager::Run 0  
OBJ: HS::FIT::Setup		
 
 RooAbsData& DataEvents::Get  Data.root MyModel 0 
RooArgSet:: = (Mmiss,fgID)
cerate weights
make data set 
[#0] WARNING:DataHandling -- RooDataSet::RooDataSet(DataEvents) WARNING: designated weight variable  not found in set of variables, no weighting will be assigned
 return DATA 
RooDataSet::DataEvents[Mmiss,fgID] = 100000 entries
 FitManager::FillEventsPDFs 2
0 0 
1 0 
RooAddPdf::TotalPDF[ Yld_Signal * Signal + Yld_BG * BG ] = 0.753265
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (Signal,BG)
Minuit2Minimizer: Minimize with max-calls 3000 convergence for edm < 1 strategy 1
MnSeedGenerator: for initial parameters FCN = -829524.

Info in <Minuit2>: Minuit2Minimizer::Hesse : Hesse is valid - matrix is accurate


[#1] INFO:Minization -- RooMinimizer::optimizeConst: deactivating const optimization

  RooFitResult: minimized FCN value: -851968, estimated distance to minimum: 1.524e-05
                covariance matrix quality: Unknown, matrix was externally provided
                Status : MINIMIZE=0 HESSE=0 HESSE=0 

    Floating Parameter    FinalValue +/-  Error   
  --------------------  --------------------------
                Yld_BG    4.9959e+04 +/-  3.58e+02
            Yld_Signal    5.0040e+04 +/-  3.59e+02
                    a0    4.5001e-01 +/-  7.55e-03
                    a1    4.6538e-03 +/-  1.13e-02
                  mean    4.9945e+00 +/-  4.22e-03
                 sigma    7.0442e-01 +/-  4.51e-03

PlotResults::PlotResults 0x7f183e8fbb50 0x7f183da2a7c0 
model 0x7f183d6ada50


Info in <Minuit2>: Minuit2Minimizer::Hesse : Hesse is valid - matrix is accurate


[#1] INFO:Plotting -- RooAbsPdf::plotOn(TotalPDF) directly selected PDF components: (Signal)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(TotalPDF) indirectly selected PDF components: ()
[#1] INFO:Plotting -- RooAbsPdf::plotOn(TotalPDF) directly selected PDF components: (BG)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(TotalPDF) indirectly selected PDF components: ()
 splot model y0x7f183d6ada50
HS::FIT::sPlot::Run create sWeights 
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#0] WARNING:Optimization -- RooAbsOptTestStatistic::optimizeConstantTerms(nll_TotalPDF_DataEvents) WARNING Cache-and-track optimization (Optimize level 2) is only available for datasets implement in terms of RooVectorDataStore - ignoring this option for current dataset
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (Signal,BG)
[#1] INFO:Fitting -- RooAbsPdf::fitTo(TotalPDF) Calculating sum-of-weights-squared cor

Warning in <TTreeFormula::DefinedVariable>: Casting to primary types like "Long64_t" is not supported yet


Weights::SortWeights() reordering trees
Weights::SortWeights() entries 100000 100000
Weights::SortWeights Print new ordering
Weights HSsWeights contains 100000 events associated file is /work/Dropbox/HaSpect/dev/HASPECT6/tutorials/RooFitExamples/WeightedObservable/sPlotFit/Weights.root 
ID branch name : fgID
Species are : 
BG
Signal
The first ten entries are :
0 0.839092 0.160913 
1 1.14794 -0.147931 
2 -0.151882 1.15188 
3 -0.438082 1.43808 
4 -0.422561 1.42256 
5 1.00834 -0.00833166 
6 -0.453222 1.45322 
7 -0.447169 1.44717 
8 0.600805 0.399199 
9 -0.449567 1.44957 
FitManager::RunOne() done 0
RooArgList:: = (mean,sigma,a0,a1)
RooArgList:: = (Yld_Signal,Yld_BG)
FiledTree::~FiledTree()  tree name MyModel 100000 Data.root
Weights::~Weights()


Weights::Save() Saved weights to file


### Draw some weighted variables

I can now use the resulting weights to make sPlots of the other variables in the tree, disentangling signal from background. 
I use the DrawWeighted function which takes standard TTree::Draw option for first argument and the weight species with the second (i.e. here I could plot Signal or BG) .

Additionally you can add cut condition with the third argument and a hist draw option with the fourth.

Here we can see the background subtracted \phi distributions for the 2 different polarisation states.

If splits were defined with LoadBinVars, then calling draw weighted automatically calls sPlot::MergeWeights() which combines all the weights into 1 file so they can be used on the full original tree. You may want to call MErgeWeights in the case you do not want to draw anything so at least the merged weights file is produced.



In [11]:
auto* can=new TCanvas;
can->Divide(2,1);
can->cd(1);
TH1F* hPhi=new TH1F("hPhi","hPhi",100,-180,180);
RF.DrawWeighted("Phi>>hPhi","Signal");
RF.DrawWeighted("Phi","Signal","PolState==-1","same");
RF.DrawWeighted("Phi","Signal","PolState==1","same");
hPhi->SetMinimum(0);

can->cd(2);
RF.DrawWeighted("M1>>hM1(100,0,10)","Signal");
RF.DrawWeighted("M1","BG","","same");

can->Draw();

Weights::SetFile /work/Dropbox/HaSpect/dev/HASPECT6/tutorials/RooFitExamples/WeightedObservable/sPlotFit//Tweights.root
Weights::Merge Merging Weights* in directory /work/Dropbox/HaSpect/dev/HASPECT6/tutorials/RooFitExamples/WeightedObservable/sPlotFit
Weights::Merge Adding file Weights.root
TFile**		/work/Dropbox/HaSpect/dev/HASPECT6/tutorials/RooFitExamples/WeightedObservable/sPlotFit/Weights.root	
 TFile*		/work/Dropbox/HaSpect/dev/HASPECT6/tutorials/RooFitExamples/WeightedObservable/sPlotFit/Weights.root	
  KEY: TTree	HSsWeights_W;1	Tree weights for each species
  KEY: TTree	HSsWeights_ID;1	event ids for each entry
  KEY: HS::Weights	HSsWeights;1	
HSsWeights 
Weights::~Weights()
Weights Add 100000 2 2 0 
FILE /work/Dropbox/HaSpect/dev/HASPECT6/tutorials/RooFitExamples/WeightedObservable/sPlotFit//Tweights.root
Weights Add new species :BG
Insert species BG 0 0
Weights Add new species :Signal
Insert species Signal 1 0
Weights::~Weights()
Weights::BuildIndex 100000
Done
Weights::SortW

Warning in <TTreeFormula::DefinedVariable>: Casting to primary types like "Long64_t" is not supported yet


Weights::Save() Saved weights to file
TFile**		/work/Dropbox/HaSpect/dev/HASPECT6/tutorials/RooFitExamples/WeightedObservable/sPlotFit/Tweights.root	
 TFile*		/work/Dropbox/HaSpect/dev/HASPECT6/tutorials/RooFitExamples/WeightedObservable/sPlotFit/Tweights.root	
  KEY: TTree	HSsWeights_W;1	Tree weights for each species
  KEY: TTree	HSsWeights_ID;1	event ids for each entry
  KEY: HS::Weights	HSsWeights;1	HSsWeights
HSsWeights HSsWeights
Weights::~Weights()
Weights::BuildIndex 100000
Done


Warning in <TTreeFormula::DefinedVariable>: Casting to primary types like "Long64_t" is not supported yet


FiledTree::~FiledTree()  tree name MyModel 100000 Data.root


Now we can proceed to the main fit and use the sWeights  with the [Fit notebook](Fit.ipynb)